# This code finds out which departures that are available to the destination that you want, AKA a Path Finder

In [20]:
import requests
import os
from dotenv import load_dotenv


load_dotenv()
API_KEY = os.getenv("TRAFFICLABS_API_KEY")

# Fetch stops for a given location
def fetch_stops(city_name):
    base_url = "https://api.resrobot.se/v2.1/location.name"
    params = {
        "input": city_name,
        "format": "json",
        "accessId": API_KEY
    }
    response = requests.get(base_url, params=params)
    
    print(f"Fetching stops for {city_name}...")
    print(f"API Response Status Code: {response.status_code}")
    print(f"API Response Content: {response.text}")

    if response.status_code == 200:
        try:
            data = response.json()
        except ValueError as e:
            print(f"Error decoding JSON: {e}")
            return []
        
        stop_data = []
        if "stopLocationOrCoordLocation" in data:
            for item in data["stopLocationOrCoordLocation"]:
                if "StopLocation" in item:
                    stop = item["StopLocation"]
                    stop_data.append({
                        "Name": stop.get("name", "Unknown"),
                        "Latitude": stop.get("lat"),
                        "Longitude": stop.get("lon"),
                        "ID": stop.get("id")
                    })
        print(f"Fetched {len(stop_data)} stops.")
        return stop_data
    else:
        print(f"Failed to fetch stops. Status Code: {response.status_code}")
        return []

# Fetch a journey/route (trip) from start to destination
def fetch_route(from_stop_id, to_stop_id):
    base_url = "https://api.resrobot.se/v2.1/trip"
    params = {
        "originId": from_stop_id,
        "destId": to_stop_id,
        "format": "json",
        "accessId": API_KEY,
        "passlist": True,
        "showPassingPoints": True
    }

    print(f"Fetching route from stop ID {from_stop_id} to {to_stop_id}...")
    response = requests.get(base_url, params=params)

    print(f"API Response Status Code: {response.status_code}")
    print(f"API Response Content: {response.text}")

    if response.status_code == 200:
        try:
            data = response.json()
        except ValueError as e:
            print(f"Error decoding JSON: {e}")
            return []

        print("Route Data:", data)  # Log the entire route data for debugging purposes

        if "Trip" in data:
            print(f"Found {len(data['Trip'])} trips.")
            return data["Trip"]
        else:
            print("No trips found between the stops.")
            return []
    else:
        print(f"Failed to fetch route. Status Code: {response.status_code}")
        return []

# Main function to orchestrate the process
# you can remove the messages if you want. I was just doing a lot of debugging to see where it gives errors
def main():
    origin_city_name = input("Enter the city you are currently in: ").strip()
    if not origin_city_name:
        print("Origin city name cannot be empty.")
        return

    print(f"Fetching stops in {origin_city_name}...")
    origin_stops = fetch_stops(origin_city_name)
    if not origin_stops:
        print(f"No stops found for {origin_city_name}. Exiting...")
        return

    print("\nAvailable stops in your city (Origin):")
    for index, stop in enumerate(origin_stops):
        print(f"{index + 1}. {stop['Name']} (ID: {stop['ID']})")

    try:
        from_stop_choice = int(input("\nSelect the stop you're at by number: ")) - 1
        if from_stop_choice < 0 or from_stop_choice >= len(origin_stops):
            print("Invalid choice. Exiting...")
            return
    except ValueError:
        print("Invalid input. Exiting...")
        return

    from_stop = origin_stops[from_stop_choice]
    from_stop_id = from_stop["ID"]
    print(f"\nYou are at: {from_stop['Name']} (ID: {from_stop_id})")

    destination_city_name = input("\nEnter the destination city: ").strip()
    if not destination_city_name:
        print("Destination city name cannot be empty.")
        return

    print(f"Fetching stops in {destination_city_name}...")
    destination_stops = fetch_stops(destination_city_name)
    if not destination_stops:
        print(f"No stops found for {destination_city_name}. Exiting...")
        return

    print(f"\nAvailable stops in {destination_city_name}:")
    for index, stop in enumerate(destination_stops):
        print(f"{index + 1}. {stop['Name']} (ID: {stop['ID']})")

    try:
        destination_stop_choice = int(input("\nSelect the destination stop by number: ")) - 1
        if destination_stop_choice < 0 or destination_stop_choice >= len(destination_stops):
            print("Invalid choice. Exiting...")
            return
    except ValueError:
        print("Invalid input. Exiting...")
        return

    destination_stop = destination_stops[destination_stop_choice]
    destination_stop_id = destination_stop["ID"]
    print(f"\nYou have selected the destination stop: {destination_stop['Name']} (ID: {destination_stop_id})")

    print(f"\nFetching route from {from_stop['Name']} to {destination_stop['Name']}...")
    route_data = fetch_route(from_stop_id, destination_stop_id)
    # you can remove this but it is there in case something happened you will know why
    if route_data:
        # Debugging route data to inspect the structure 
        print("\nRoute details:")  # Start printing the route details
        for connection in route_data:
            print(f"Connection Data: {connection}")  # Print the entire connection to inspect its structure
            
            # Now check if the keys 'from' and 'to' exist in the connection, and print based on that
            if 'from' in connection and 'to' in connection:
                print(f"Connection from {connection['from']['name']} to {connection['to']['name']}")
                for leg in connection.get("LegList", {}).get("Leg", []):
                    transport = leg.get("Product", {}).get("name", "Unknown")
                    departure_time = leg.get("Origin", {}).get("dateTime", "Unknown")
                    arrival_time = leg.get("Destination", {}).get("dateTime", "Unknown")
                    origin = leg.get("Origin", {}).get("name", "Unknown")
                    destination = leg.get("Destination", {}).get("name", "Unknown")
                    line = leg.get("Product", {}).get("line", "Unknown")

                    print(f"Take {transport} {line} from {origin} to {destination}.")
                    print(f"Departure: {departure_time}, Arrival: {arrival_time}")
                    print("-" * 40)
            else:
                print("No valid connection found in this route.")
    else:
        print("No route data available.")


if __name__ == "__main__":
    main()


Fetching stops in Partille...
Fetching stops for Partille...
API Response Status Code: 200
API Response Content: {"stopLocationOrCoordLocation":[{"StopLocation":{"productAtStop":[{"icon":{"res":"prod_gen"},"cls":"16"},{"icon":{"res":"prod_gen"},"cls":"128"}],"timezoneOffset":60,"id":"A=1@O=PARTILLE@X=12102861@Y=57740870@U=1@L=740098044@B=1@p=1738039011@","extId":"740098044","name":"PARTILLE","lon":12.102861,"lat":57.74087,"weight":1711,"products":144,"meta":true,"minimumChangeDuration":"PT26M"}},{"StopLocation":{"productAtStop":[{"icon":{"res":"prod_gen"},"cls":"16"},{"icon":{"res":"prod_gen"},"cls":"128"}],"timezoneOffset":60,"id":"A=1@O=Partille station@X=12102861@Y=57740870@U=1@L=740000132@B=1@p=1738039011@","extId":"740000132","name":"Partille station","lon":12.102861,"lat":57.74087,"weight":1711,"products":144,"minimumChangeDuration":"PT5M"}},{"StopLocation":{"productAtStop":[{"icon":{"res":"prod_gen"},"cls":"128"}],"timezoneOffset":60,"id":"A=1@O=Partille arena@X=12118016@Y=57741